In [1]:
from pyspark.sql.functions import coalesce
from pyspark.sql import SparkSession

In [2]:
realnews_df = spark.read.json("/common/users/ac1771/realnews/realnews.jsonl")

In [3]:
uci_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/news-aggregator-dataset/uci-news-aggregator.csv", header=True)
abc_news_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/a-million-news-headlines/abcnews-date-text.csv", header=True)
all_the_news_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/all-the-news-2/all-the-news-2-1.csv", header=True)
india_df = spark.read.csv("/common/users/shared/cs543_fall22_group3/india-headlines-news-dataset/india-news-headlines.csv", header=True)
news_cat = spark.read.json("/common/users/shared/cs543_fall22_group3/news-category-dataset/News_Category_Dataset_v3.json")

In [4]:
print(realnews_df.count() + uci_df.count() + abc_news_df.count() + all_the_news_df.count() + india_df.count() + news_cat.count())

42183620


In [6]:
news_cat.show()

+--------------------+--------------+----------+--------------------+--------------------+--------------------+
|             authors|      category|      date|            headline|                link|   short_description|
+--------------------+--------------+----------+--------------------+--------------------+--------------------+
|Carla K. Johnson, AP|     U.S. NEWS|2022-09-23|Over 4 Million Am...|https://www.huffp...|Health experts sa...|
|      Mary Papenfuss|     U.S. NEWS|2022-09-23|American Airlines...|https://www.huffp...|He was subdued by...|
|       Elyse Wanshel|        COMEDY|2022-09-23|23 Of The Funnies...|https://www.huffp...|"Until you have a...|
|    Caroline Bologna|     PARENTING|2022-09-23|The Funniest Twee...|https://www.huffp...|"Accidentally put...|
|      Nina Golgowski|     U.S. NEWS|2022-09-22|Woman Who Called ...|https://www.huffp...|Amy Cooper accuse...|
|                    |     U.S. NEWS|2022-09-22|Cleaner Was Dead ...|https://www.huffp...|The 63-year-ol

In [8]:
news_cat = news_cat.select("*", coalesce(news_cat["short_description"], news_cat["headline"]).alias("selected_text"))
news_cat = news_cat.select("selected_text")

In [9]:
realnews_df = realnews_df.select("*", coalesce(realnews_df["summary"], realnews_df["title"]).alias("selected_text"))
realnews_df = realnews_df.select("selected_text")

In [13]:
all_the_news_df.show()

+--------------------+--------------------+----------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                date|                year|     month|             day|              author|               title|             article|                 url|             section|         publication|
+--------------------+--------------------+----------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 2016-12-09 18:31:00|                2016|      12.0|               9|         Lee Drutman|We should take co...|"This post is par...|             however| several critics ...|         for example|
| 2016-10-07 21:26:46|                2016|      10.0|               7|         Scott Davis|Colts GM Ryan Gri...| The Indianapolis...|                null|                null|                null|
|The highe

In [14]:
uci_df = uci_df.select("TITLE").withColumnRenamed("TIMESTAMP","publish_date").withColumnRenamed("TITLE", "selected_text")
abc_news_df = abc_news_df.select("headline_text").withColumnRenamed("headline_text", "selected_text")
india_df = india_df.select("headline_text").withColumnRenamed("headline_text", "selected_text")
all_the_news_df = all_the_news_df.select("title").withColumnRenamed("title", "selected_text")

In [15]:
combined_df = realnews_df.unionByName(uci_df).unionByName(abc_news_df).unionByName(india_df).unionByName(all_the_news_df).unionByName(news_cat)

In [17]:
# print('Total Row Count: {}'.format(combined_df.count()))
combined_df.show()

+--------------------+
|       selected_text|
+--------------------+
|Pretzel sticks, p...|
|We Catholics have...|
|A woman who alleg...|
|It’s a huge bonus...|
|Little lad, two, ...|
|The five highest ...|
|If Vegas had hand...|
|Ed Gillespie is t...|
|Migrants fleeing ...|
|Somewhere in the ...|
|Minute-by-minute ...|
|GOP candidates ba...|
|Moussa Abu Marzou...|
|Los Angeles-based...|
|A jug containing ...|
|Pregnant Cheryl g...|
|Nicole Richie is ...|
|Colorado's Denver...|
|Brition Alexandra...|
|The number of wet...|
+--------------------+
only showing top 20 rows



In [18]:
combined_df.write.mode("Overwrite").json('/common/users/shared/cs543_fall22_group3/combined/combined_raw')